## Problem 3

Consider the case where we have two matrix A and B. We want to multiply them together.

Consider two matrices:
  $$
  A = \begin{pmatrix}
  1 & 2 & 3 \\
  4 & 5 & 6
  \end{pmatrix}, 
  \quad
  B = \begin{pmatrix}
  7 & 8 \\
  9 & 10 \\
  11 & 12
  \end{pmatrix}
  $$
  The product \( C = AB \) is:
  $$
  C = \begin{pmatrix}
  (1 \times 7 + 2 \times 9 + 3 \times 11) & (1 \times 8 + 2 \times 10 + 3 \times 12) \\
  (4 \times 7 + 5 \times 9 + 6 \times 11) & (4 \times 8 + 5 \times 10 + 6 \times 12)
  \end{pmatrix}
  $$
  Simplified, this gives:
  $$
  C = \begin{pmatrix}
  58 & 64 \\
  139 & 154
  \end{pmatrix}
  $$
  

In [1]:
def matrix_product(A, B):
    # Get dimensions of matrices
    m, n = len(A), len(A[0])
    n_b, p = len(B), len(B[0])
    
    # Check if the matrices can be multiplied
    if n != n_b:
        raise ValueError("Number of columns in A must equal number of rows in B")
    
    # Initialize the resulting matrix with zeros
    C = [[0] * p for _ in range(m)]
    
    # Perform matrix multiplication
    for i in range(m):
        for j in range(p):
            for k in range(n):
                C[i][j] += A[i][k] * B[k][j]
    
    return C


In [2]:
import numpy as np
# Example matrices
A = [[1, 2, 3],
    [4, 5, 6]]

B = [[7, 8],
    [9, 10],
    [11, 12]]

print("Matrix Product:", matrix_product(A, B))

A = np.array(A)
B = np.array(B)

# Output the result
print("Matrix Product:", A @ B)

Matrix Product: [[58, 64], [139, 154]]
Matrix Product: [[ 58  64]
 [139 154]]


Which one is better?
Let's see how much time does it takes. 

In [3]:
import time

TimesN = 10
size = 128

np_execution_time = 0
mp_execution_time = 0
for _ in range(TimesN):
    A = np.random.rand(size, size)
    B = np.random.rand(size, size)
    
    # Measure the execution time of numpy matrix product
    np_start_time = time.time()
    result_np = A @ B
    np_end_time = time.time()
    np_execution_time += np_end_time - np_start_time

    # Measure the execution time of matrix product function
    mp_start_time = time.time()
    result_mp = matrix_product(A, B)
    mp_end_time = time.time()
    mp_execution_time = mp_end_time - mp_start_time

# Print the execution times
print("Numpy Execution Time:", np_execution_time/TimesN)
print("Matrix Product Execution Time:", mp_execution_time/TimesN)

Numpy Execution Time: 0.002417111396789551
Matrix Product Execution Time: 0.05779907703399658


## Problem 4

It seems like our method still have space to improve. How?

Consider this case

$A=\left[\begin{array}{ll}a & b \\ c & d\end{array}\right], B=\left[\begin{array}{ll}e & f \\ g & h\end{array}\right], C=A B=\left[\begin{array}{ll}C_{11} & C_{12} \\ C_{21} & C_{22}\end{array}\right]$

We have $C_{11}=a e+b g, C_{12}=a f+b h, C_{21}=c e+d g, C_{22}=c f+d h$


In [4]:
def split(M):
    # Get the number of rows and columns
    n = len(M)
    return M[:n//2, :n//2], M[:n//2, n//2:], M[n//2:, :n//2], M[n//2:, n//2:]

In [5]:
A = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]])
a, b, c, d = split(A)
print("a:", a)
print("b:", b)
print("c:", c)
print("d:", d)

a: [[1 2]
 [5 6]]
b: [[3 4]
 [7 8]]
c: [[ 9 10]
 [13 14]]
d: [[11 12]
 [15 16]]



$a e+b g=(a+d)(e+h)+d(g-e)-(a+b) h+(b-d)(g+h) $

$a f+b h=a(f-h)+(a+b) h $

$c e+d g=(c+d) e+d(g-e) $

$c f+d h=a(f-h)+(a+d)(e+h)-(c+d) e-(a-c)(e+f)$

Can we find any pattern here?

In [25]:
# Strassen algorithm
def strassen(A, B):
    n = len(A)
    if n <= 2:  # Base case
        return np.dot(A, B)
    
    a,b,c,d = split(A)
    e,f,g,h = split(B)
    
    p1 = strassen(a+d, e+h)
    p2 = strassen(d, g-e)
    p3 = strassen(a+b, h)
    p4 = strassen(b-d, g+h)
    p5 = strassen(a, f-h)
    p6 = strassen(c+d, e)
    p7 = strassen(a-c, e+f)
    
    c11 = p1 + p2 - p3 + p4
    c12 = p5 + p3
    c21 = p6 + p2
    c22 = p5 + p1 - p6 - p7
    
    C = np.vstack((np.hstack((c11, c12)), np.hstack((c21, c22))))
    return C

In [26]:
A = np.random.rand(2, 2)
B = np.random.rand(2, 2)

C_s = strassen(A, B)
C_np = A @ B
print("Strassen:", C_s)
print("Numpy:", C_np)

Strassen: [[0.4259333  0.3205607 ]
 [0.44354849 0.33260736]]
Numpy: [[0.4259333  0.3205607 ]
 [0.44354849 0.33260736]]


Did we really improve the performance?

In [28]:
TimesN = 1
size = 128

strassen_execution_time = 0

for _ in range(TimesN):
    A = np.random.rand(size, size)
    B = np.random.rand(size, size)
    
    # Measure the execution time of strassen matrix product
    strassen_start_time = time.time()
    result_strassen = strassen(A, B)
    strassen_end_time = time.time()
    strassen_execution_time += strassen_end_time - strassen_start_time

    # Measure the execution time of matrix product function
    mp_start_time = time.time()
    result_mp = matrix_product(A, B)
    mp_end_time = time.time()
    mp_execution_time = mp_end_time - mp_start_time

# Print the execution times
print("Strassen Execution Time:", strassen_execution_time/TimesN)
print("Matrix Product Execution Time:", mp_execution_time/TimesN)

Strassen Execution Time: 0.3918471336364746
Matrix Product Execution Time: 0.5195739269256592


We do make some difference here. What is its complexity?